<a href="https://colab.research.google.com/github/supun-kanda/advanced-cache-replacement/blob/main/existing_algorithm_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

Datasets
1.   Caffiene Block cache trace: web07 (uploaded)
2.   Twitter Cluster based file cache: cluster052




In [12]:
path_web07 = '/content/web07.trace'
path_web12 = '/content/web12.trace'
path_cluster052 = 'https://raw.githubusercontent.com/twitter/cache-trace/master/samples/2020Mar/cluster052'

Load web07 cache

In [3]:
with open(path_web07, 'rb') as file:
    data = file.read(4) # integer = 4 x bytes
    web07 = []
    while data:
      web07.append(int.from_bytes(data, "big"))
      data = file.read(4)

In [13]:
with open(path_web12, 'rb') as file:
    data = file.read(4) # integer = 4 x bytes
    web12 = []
    while data:
      web12.append(int.from_bytes(data, "big"))
      data = file.read(4)

Block Cache Dataset Totals Keys vs Unique Keys

In [4]:
print(len(web07),'->',len(set(web07)))

76118 -> 20484


In [14]:
print(len(web12),'->',len(set(web12)))

95607 -> 13756


In [5]:

columns=['time','key','key_size','value_size','client_id','operation','TTL']
cluster052 = pd.read_csv(path_cluster052, sep=',', on_bad_lines='skip', names=columns)

Page Cache Dataset Totals Keys vs Unique Keys

In [6]:
print(cluster052['key'].count(),'->',cluster052['key'].unique().size)

1000000 -> 255333


In [15]:
import random

from lru_cache import LRUCache

def generate_workload(num_requests, num_unique_keys):
    workload = []
    keys = list(range(num_unique_keys))
    for _ in range(num_requests):
        key = random.choice(keys)
        workload.append(key)
    return workload

def cache_benchmark(cache, workload):
    hits = 0
    misses = 0
    for key in workload:
        if cache.get(key) is not None:
            hits += 1
        else:
            cache.put(key, key)
            misses += 1
    hit_rate = hits / len(workload)
    miss_rate = misses / len(workload)
    return hit_rate, miss_rate

# Example usage
cache_capacity = 3000
num_requests = 95607
num_unique_keys = 13756

cache = LRUCache(cache_capacity)
# workload = generate_workload(num_requests, num_unique_keys)

hit_rate, miss_rate = cache_benchmark(cache, web12)

print("Cache hit rate:", hit_rate)
print("Cache miss rate:", miss_rate)

Cache hit rate: 0.7648498540901817
Cache miss rate: 0.2351501459098183
